# Managing Conversation History


One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.


Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

We can do this by adding a simple step in front of the prompt that modifies the ```messages``` key appropriately, and then wrap that new chain in the Message History class.


LangChain comes with a few built-in helpers for managing a list of messages. 

https://python.langchain.com/v0.2/docs/how_to/#messages


 In this case we'll use the trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:
https://python.langchain.com/v0.2/docs/how_to/trim_messages/



In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="3.ChatBot_with_conversation_history_management"

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [4]:
# Initialise empty messages list
messages = []

# Let's add it to our chain

To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.



In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [6]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


In [7]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [8]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

In [9]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [10]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [11]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

In [12]:
config_1 = {"configurable": {"session_id": session_id_1}}

In [13]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Hi Todd! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 33, 'total_tokens': 43}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f3db212e1c', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ec1df2f-c077-43ab-a503-292d1dc67c9e-0')

In [14]:
store

{'12265e5e-5750-43ed-acb3-afbe010edff3': InMemoryChatMessageHistory(messages=[HumanMessage(content="hi! I'm todd"), AIMessage(content='Hi Todd! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 33, 'total_tokens': 43}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f3db212e1c', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ec1df2f-c077-43ab-a503-292d1dc67c9e-0')])}

### It knows the answer since the trimming for now hasn't removed the history...

In [15]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your name is Todd. How can I help you today, Todd?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 56, 'total_tokens': 70}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-2bc6d6e9-f02d-4d8f-8026-8dc899256b1d-0')

### It doesn't know about any math problem...

In [16]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config_1,
)

response.content

"You haven't asked any math problems yet. If you have a math question or problem you'd like help with, feel free to share it!"

### It still knows the answer since the trimming for now hasn't removed the history...

In [17]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content='You mentioned your name is Todd. How can I assist you further, Todd?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 103, 'total_tokens': 119}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-ee2f1ccf-eb10-4ae8-bea9-f0ac6b5228a9-0')

# It's Lost due to trimming !

In [18]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content="I'm sorry, but I don't have access to personal information unless you've shared it in this conversation. You can let me know your name if you'd like!", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 105, 'total_tokens': 135}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-875955e9-9ea8-4662-a22c-587480e337be-0')

# Put it back in memory !

In [24]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="My name is John!?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content='Nice to meet you, John! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 84, 'total_tokens': 98}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e35d0a3-0a4f-4601-88f1-41afc41f5e1b-0')

In [25]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content='Your name is John. How can I help you today, John?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 111, 'total_tokens': 125}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-f62897fc-bca3-4388-a813-35cd14bc4935-0')

In [26]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content='Your name is John. If you have any other questions or need assistance, feel free to ask!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 88, 'total_tokens': 108}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f3db212e1c', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b856272-9748-40e7-b701-388bfb9a6e42-0')

In [27]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content='Your name is John. Is there anything else you would like to know?', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 121, 'total_tokens': 136}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf7b70b5-5e74-44bc-b8b5-b7d3b4ea5cb6-0')

# It's Lost again due to trimming yet again !

In [28]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config_1,
)

response

AIMessage(content="I'm sorry, but I don't have access to personal information, including your name, unless you tell me. If you'd like to share your name or have any other questions, feel free to let me know!", response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 122, 'total_tokens': 163}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-943c765e-412e-4595-9634-4f2cd0f19704-0')